# Section 1 Install some stuff an load data
*PLACEHOLDER*

In [7]:
# Install dependencies as needed:
# Add your installations here please :)
# Install dependencies based on OS
import os
import sys
if sys.platform == "win32":
    %pip install kagglehub[pandas-datasets] plotly
else:
    %pip install "kagglehub[pandas-datasets]" "plotly"


Note: you may need to restart the kernel to use updated packages.


In [13]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from collections import OrderedDict

In [8]:
# Set the path to the file you'd like to load
file_paths = ["high_popularity_spotify_data.csv", "low_popularity_spotify_data.csv"]
database = []

# Load the latest version
for file_path in file_paths:
    df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "solomonameh/spotify-music-dataset",
    file_path,
    # Provide any additional arguments like 
    # sql_query or pandas_kwargs. See the 
    # documenation for more information:
    # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
    )
    database.append(df)

print("First 5 records:", df.head())

/tmp/ipykernel_12953/3214104888.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(
/tmp/ipykernel_12953/3214104888.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:    time_signature  track_popularity  speechiness  danceability  playlist_name  \
0             4.0                23       0.0393         0.636  Rock Classics   
1             4.0                53       0.0317         0.572  Rock Classics   
2             4.0                55       0.0454         0.591  Rock Classics   
3             4.0                64       0.1010         0.443  Jazz Classics   
4             4.0                62       0.0298         0.685  Jazz Classics   

                       track_artist  duration_ms  energy playlist_genre  \
0      Creedence Clearwater Revival     138053.0   0.746           rock   
1                         Van Halen     241600.0   0.835           rock   
2                      Stevie Nicks     329413.0   0.804           rock   
3  Ella Fitzgerald, Louis Armstrong     185160.0   0.104           jazz   
4                    Galt MacDermot     205720.0   0.472           jazz   

  playlist_subgenre  ...          track_album

# Some Data wrangling I guess

Lil section for preprocessing. 

This is where we do the data stuff. I wrote commends read them sometimes they make sense



In [9]:
# Everything that needs to be done in the main DataFrame goes in this cell


spotify_combined = pd.concat(database, ignore_index=True)
print("Combined DataFrame shape:", spotify_combined.shape)

#Do all cleaning steps here
#Convert that to dates
spotify_combined["track_album_release_date"] = pd.to_datetime(spotify_combined["track_album_release_date"], format = "mixed")


#Feature engineering goes into the main Dataframe
spotify_combined['year'] = spotify_combined['track_album_release_date'].dt.year
spotify_combined['decade'] = (spotify_combined['year'] // 10) * 10


Combined DataFrame shape: (4831, 29)


In [17]:
# Figure Data goes in this cell
# All data manipulations that have to be done based on user selection go into the cell the figure code is in. 

# A dict of dataframes to put all the figure data into

# We can use this to display different figures based on user selection later
## Also give it a discriptive name that will match user dropdown selection.
figure_data = OrderedDict()

#Figure 1
decade_genre_counts = spotify_combined.groupby(['decade', 'playlist_genre']).size().reset_index(name='count')
figure_data["produced_count"] = decade_genre_counts

decade_genre_counts_weighted = (
    spotify_combined
    .groupby(['decade', 'playlist_genre'])['track_popularity']
    .sum()
    .reset_index(name='count')
)

figure_data["popularity_count"] = decade_genre_counts

# Figure 1 - Genres per decade displaying all Decades
I dont know yet what figure 1 Is but ill figure it out.

In [23]:
# Most produced Genres per decade as barplot
# Display change from one decade to previous and next by grouping decades in trios 
# differnetiate with colors
# repeat for popular songs to get most popular genres per decade


# Create bar plot
# This is for later dummy to make interactive with user input
user_data_selection = "popularity_count"
user_amount_genres = 3

data = figure_data[user_data_selection]
top_per_decade = (
    data.sort_values(["decade", "count"], ascending=[True, False])
    .groupby("decade")
    .head(user_amount_genres)
)
fig = px.bar(
    top_per_decade,
    x="decade",
    y="count",
    color="playlist_genre",
    title=f"Top {user_amount_genres} genres per decade",
    labels={"decade": "Decade", "count": f"{user_data_selection} of Songs", "playlist_genre": "Genre"},
    barmode="group",
)
fig.show()


# Figure 2 - Genre per decade only displaying selected deacde
with change from previous to next

In [ ]:
import plotly.express as px

data_selection = "popularity_count"
amount_genres_selection = 3
decade_selection = 1990

data = figure_data[user_data_selection]

# Get data for three decades
data_current = data[data['decade'] == decade_selection].copy()
data_prev = data[data['decade'] == decade_selection - 10].copy()
data_next = data[data['decade'] == decade_selection + 10].copy()

# Get top genres from current decade
top_genres = (
    data_current.nlargest(amount_genres_selection, 'count')['playlist_genre'].unique()
)

# Add decade labels and combine
data_current['decade_label'] = f"{decade_selection}s"
data_prev['decade_label'] = f"{decade_selection - 10}s"
data_next['decade_label'] = f"{decade_selection + 10}s"

combined = pd.concat([data_prev, data_current, data_next], ignore_index=True)
combined = combined[combined['playlist_genre'].isin(top_genres)]

fig = px.bar(
    combined,
    x='playlist_genre',
    y='count',
    color='decade_label',
    barmode='group',
    title=f"Top {amount_genres_selection} genres across decades",
    labels={'count': 'Count', 'playlist_genre': 'Genre', 'decade_label': 'Decade'},
)
fig.show()


# Figure n ...